# Local Adressability with DMM

In [ ]:
from dataclasses import replace
from pulser.channels.dmm import DMM
from pulser.devices import Chadoq2, MockDevice
from pulser.register import Register
from pulser.register.register_layout import RegisterLayout
from pulser.register.mappable_reg import MappableRegister
from pulser.register.weight_maps import DetuningMap
from pulser.sequence import Sequence
from pulser.pulse import Pulse
from pulser.waveforms import ConstantWaveform

## Basics

Even when working with global addressing channels, the detuning of individual qubits can be addressed locally by using a specific channel named the Detuning Map Modulator or `DMM`. This `Channel` applies a `Global` pulse of zero amplitude and negative detuning on a `DetuningMap`. The `DetuningMap` consists of a set of weights on specific sites that dictate how the detuning applied by the `DMM` is distributed.

This modulation of the `DetuningMap` by the `DMM` Channel is equivalent to adding a term 
$ - \frac{\hbar}{2} \sum_i \epsilon_i \Delta(t) \sigma^z_i$ to the Ising Hamiltonian. Here, $\Delta(t)$ is the modulating detuning sent on the `DMM`, and $(\epsilon_i)_{i}$ are the weights defined in the `DetuningMap` for each atom.

## Define a Detuning Map

A `DetuningMap` associates a set of locations with a set of weights. The weights $(\epsilon_i)_i$ have to be positive, between 0 and 1, and their sum has to be equal to 1. The locations are the trap coordinates to address.

In [ ]:
trap_coordinates = [(0.0, 0.0), (0.0, 4.0), (4.0, 0.0), (4.0, 4.0)]
weights = [0.5, 0.25, 0.25, 0]  # between 0 and 1, sum equal to 1

The `DetuningMap` can be defined from a `RegisterLayout`, a `Register` or a `MappableRegister` by using the `define_detuning_map` method. All it takes is a mapping between trap/qubit IDs and weights.

In [ ]:
register_layout = RegisterLayout(trap_coordinates)
det_map_from_reg_layout = register_layout.define_detuning_map(
    {i: weights[i] for i in range(4)}  # mapping between trap ids and weights
)

map_reg = MappableRegister(register_layout)
det_map_from_map_reg = map_reg.define_detuning_map(
    {i: weights[i] for i in range(4)}  # mapping between trap ids and weights
)

register = Register.from_coordinates(
    trap_coordinates, center=False, prefix="q"
)
det_map_from_reg = register.define_detuning_map(
    {
        f"q{i}": weights[i] for i in range(4)
    }  # mapping between qubit ids and weights
)

det_map_from_reg_layout.draw(labels=[0, 1, 2, 3])
assert det_map_from_reg_layout == det_map_from_map_reg
assert det_map_from_reg_layout == det_map_from_reg

The traps are represented by a green dot and highlighted by a grey square, whose shade is proportional to the value of the associated weight. It is possible to display the value of the weights by specifying a list of `labels`. If the weight is null then no `DMM` is applied: the atom is not highlighted by a grey square and the value of the weight is not shown when `labels` are specified.

## DMM Channel and Device

A `DMM` Channel is a `Channel` that accepts pulses of zero amplitude and detuning below 0 and above `bottom_detuning`:

In [ ]:
dmm = DMM(
    clock_period=4,
    min_duration=16,
    max_duration=2**26,
    bottom_detuning=-20,  # detuning between 0 and -20 rad/µs
)

The `DMM` channels are defined separately from the other channels of the device. They are defined as a list in `dmm_objects`. They can be accessed via the property `dmm_channels` of the device, where an id is automatically associated to each `DMM` with the format `dmm_{index of appearance in dmm_objects}`.

In [ ]:
mymockdevice = replace(MockDevice, dmm_objects=(dmm, DMM()))
print(mymockdevice.dmm_channels)

## DMM in a Sequence

In a `Sequence` defined with a `Device` having `dmm_objects`, the `DMM` channels can be configured using `config_det_map`. This configuration associates a `DMM` - referenced by `dmm_id` in the `Device`- with a `DetuningMap`, under a `dmm_name` in the `Sequence`.

In [ ]:
seq = Sequence(register, mymockdevice)
seq.config_detuning_map(detuning_map, "dmm_0")
print(seq.declared_channels)

The `dmm_name` is generated automatically. it is `dmm_id` if no detuning map has been associated to the `DMM` referenced by `dmm_id` in the device, otherwise it takes the suffix `_{number of times dmm_id has been declared so far}`. For instance, if we associate a new detuning map to `dmm_0` in the previous sequence, the name of this new declared channel will be `dmm_0_1`. To be noted that configuring a `dmm_id` multiple times is only possible in a `VirtualDevice` with `reusable_channels=True`.

In [ ]:
seq.config_detuning_map(detuning_map, "dmm_0")
print(seq.declared_channels.keys())

Let's now modulate the detuning maps. This is done by calling `modulate_det_map` and providing a waveform (the detuning $\Delta(t)$ to modulate the `DetuningMap` with) and the `dmm_name` to use (provides the `DetuningMap` and checks that $\Delta(t)$ is in agreement with the characteristics of the corresponding `DMM`).

In [ ]:
seq.modulate_det_map(ConstantWaveform(100, -5), "dmm_0")
seq.modulate_det_map(ConstantWaveform(100, -10), "dmm_0_1")
seq.draw()

Once a detuning map is configured, it is not possible to define a `Microwave` channel. However, it is possible to declare `Rydberg` and `Raman` channels. Let's declare a `Rydberg.Global` channel in our sequence.

Be careful with the protocols that are used to add the pulses to `Channels` and `DMM`:
- The protocol to modulate a detuning map is by default `"no-delay"`, meaning that it will be added straight after the last pulse, without caring about the other channels. This is why the pulse on `dmm_0_1` is added at $t=0$. It can be changed to other values by defining the `protocol` argument.
- The protocol to add a pulse on `Rydberg` or a `Raman` channel is `"min-delay"`, meaning that the pulse will by default be added after the end of the pulse of a `DMM`. It can be switched to `"no-delay"`.

In [ ]:
seq.declare_channel("ryd_glob", "rydberg_global")
seq.add(
    Pulse.ConstantPulse(100, 10, 0, 0), "ryd_glob"
)  # added after the two DMM pulses
seq.modulate_det_map(ConstantWaveform(200, -10), "dmm_0")  # added at t=100
seq.add(
    Pulse.ConstantPulse(100, 5, 0, 0), "ryd_glob", protocol="no-delay"
)  # added at t=200
seq.draw()

## DMM and SLM Mask

To support an SLM Mask, a device needs to have at least a `DMM` channel defined in its `dmm_objects`. `DMM` channels are used to configure an SLM Mask, you can find more information about this relationship in the tutorial on [preparing a state using an SLM Mask](slm_mask.nblink).